# Config and Setup

## Imports

In [1]:
# utils
from google.colab import drive
import os
import random
from PIL import Image
import matplotlib.pyplot as plt
import pdb
import numpy as np
import pandas as pd

# torch
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch
from torch.utils.data import Dataset, ConcatDataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.utils.data import random_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.io import read_image

# sklearn
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score

## Config

In [2]:
# pytorch gpu config
seed = 0
torch.manual_seed(seed)
if torch.cuda.is_available():
  device = torch.device("cuda")
  torch.cuda.manual_seed_all(seed)
else:
  device = torch.device("cpu")
print(device)

cuda


## Downloading and Extracting Data
The creation of these datasets is described in the following notebooks:
*   [DiffusionDB Data](https://github.com/JensenZhaoUT/MIE1517_Project/blob/main/EDA%20-%20Generated.ipynb)
*   [WikiArt Data](https://github.com/JensenZhaoUT/MIE1517_Project/blob/main/WikiArt%20EDA.ipynb)



In [3]:
# download the datasets to working directory
!gdown 1piD2RMptZCkudbRLXxjQ2LDWA62Gw-nR
!gdown 1MqvNIAAFaArWyXg8p_egMZkCuh6uhM0q
# create directories for data and unzip archives
!mkdir /content/diffusiondb
!mkdir /content/wikiart
!unzip /content/cropped_diffusiondb.zip -d /content/diffusiondb
!unzip /content/cropped_paintings.zip -d /content/wikiart

Streaming output truncated to the last 5000 lines.
 extracting: /content/diffusiondb/cropped_image_10090.json  
 extracting: /content/diffusiondb/cropped_image_10091.jpg  
 extracting: /content/diffusiondb/cropped_image_10091.json  
 extracting: /content/diffusiondb/cropped_image_10092.jpg  
 extracting: /content/diffusiondb/cropped_image_10092.json  
 extracting: /content/diffusiondb/cropped_image_10093.jpg  
 extracting: /content/diffusiondb/cropped_image_10093.json  
 extracting: /content/diffusiondb/cropped_image_10094.jpg  
 extracting: /content/diffusiondb/cropped_image_10094.json  
 extracting: /content/diffusiondb/cropped_image_10095.jpg  
 extracting: /content/diffusiondb/cropped_image_10095.json  
 extracting: /content/diffusiondb/cropped_image_10096.jpg  
 extracting: /content/diffusiondb/cropped_image_10096.json  
 extracting: /content/diffusiondb/cropped_image_10097.jpg  
 extracting: /content/diffusiondb/cropped_image_10097.json  
 extracting: /content/diffusiondb/cropped

## Creating Custom Dataset

In [4]:
class CustomImageDataset(Dataset):
    def __init__(self, directory, label, max_images=None):
        self.directory = directory
        self.label = label
        self.images = [file for file in os.listdir(directory) if file.endswith('.jpg')]
        self.transform = transforms.ToTensor()  # Define the transform here

        # If max_images is set, randomly select a subset of images
        if max_images is not None and max_images < len(self.images):
            self.images = random.sample(self.images, max_images)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.directory, self.images[idx])
        image = Image.open(img_path)
        image = self.transform(image)  # Transform the image to a tensor
        # load image with gpu if available
        if torch.cuda.is_available():
            image = image.to(torch.device('cuda:0'))
        return image, self.label


# Determine the number of images in the smaller dataset
num_images_in_smaller_dataset = len([file for file in os.listdir('/content/diffusiondb') if file.endswith('.jpg')])

# Create dataset with true and false labels
dataset_false = CustomImageDataset('/content/diffusiondb', 0)
dataset_true = CustomImageDataset('/content/wikiart', 1, max_images=num_images_in_smaller_dataset) #since wikiart has more samples, set max

# Concat datasets into one
combined_dataset = ConcatDataset([dataset_false, dataset_true])

In [5]:
# Train-Validation-Test Split
length_of_combined_dataset = len(combined_dataset)

# define split size
train_size = int(0.7 * length_of_combined_dataset)  # 70% for training
val_size = int(0.1 * length_of_combined_dataset)   # 10% for validation
test_size = length_of_combined_dataset - train_size - val_size  # Remaining 20% for testing

# perform the split
train_dataset, val_dataset, test_dataset = random_split(combined_dataset, [train_size, val_size, test_size])

# Model

## Model Definition

In [13]:
class MyConvNet(nn.Module):
  def __init__(self, lr=0.01, criterion=nn.BCELoss):
    super(MyConvNet, self).__init__()

    # define params
    self.lr = lr
    self.criterion = criterion()


    # build the encoder and classifier
    self.encoder = self.build_encoder()
    self.classifier = self.build_classifier()


  def build_encoder(self):
    encoder = nn.Sequential(
      nn.Conv2d(in_channels = 3, out_channels = 9, kernel_size = 3, stride = 2, padding = 0), # (9,255,255)
      nn.ReLU(),
      nn.MaxPool2d(kernel_size = 4, stride = 1, padding = 0), # (9, 252, 252)
      nn.Conv2d(in_channels = 9, out_channels = 12, kernel_size = 9, stride = 1, padding = 0), #(12,244,244)
      nn.ReLU(),
      nn.MaxPool2d(kernel_size = 4, stride = 1 , padding = 0), # (12, 241, 241)
      )
    return encoder

  def build_classifier(self):
    classifier = nn.Sequential(
      nn.Flatten(start_dim=1),
      nn.Linear(12*241*241, 512),
      nn.ReLU(),
      nn.Linear(512, 1),
      nn.Sigmoid()
    )
    return classifier


  def forward(self, img):
    return self.classifier(self.encoder(img.to(torch.float)))

  def train(self, data, val_data, n_epochs=20, batch_size=15, optimizer=torch.optim.Adam):
    # read params and assign
    self.batch_size = batch_size
    train_loader = DataLoader(dataset=data, shuffle=True, batch_size=self.batch_size)
    val_loader = DataLoader(dataset=val_data, shuffle=True, batch_size=self.batch_size)

    # define optimizer
    self.optimizer = optimizer(self.parameters(), lr = self.lr)
    n_training_steps = len(train_loader)
    n_val_steps = len(val_loader)


    train_losses_epochs = []
    val_losses_epochs = []

    for epoch in range(n_epochs):
      output = []
      batch_accuracies = []
      train_losses = []
      # training loop
      for i, (x_train, y_train) in enumerate(train_loader):
        # fix typing of labels
        y_train = y_train.type(torch.FloatTensor).unsqueeze(1)
        # make sure we are using cuda
        if torch.cuda.is_available():
          x_train = x_train.to(torch.device('cuda:0'))
          y_train = y_train.to(torch.device('cuda:0'))
        # forward step
        y_pred = self.forward(x_train)
        # get loss
        loss = self.criterion(y_pred, y_train)

        # backprop
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        # add outputs to list
        output.append(y_pred)
        # add loss to list
        train_losses.append(loss.item())

        # get accuracy
        correct = (y_pred.round() == y_train).type(torch.float).sum().item()
        accuracy_batch = 100*correct/len(x_train)
        batch_accuracies.append(accuracy_batch)

        # print batch info
        if (i+1 == n_training_steps):
          avg_loss = sum(train_losses)/len(train_losses)
          # add average loss to training losses
          train_losses_epochs.append(avg_loss)
          # get accuracy of epoch
          accuracy = sum(batch_accuracies)/len(batch_accuracies)
          # print current epoch info
          print(f'Epoch {epoch+1}/{n_epochs} : Train Accuracy: {accuracy:.2f}%, Avg Loss: {avg_loss:.4f}')


      # validation loop
      batch_accuracies = []
      val_losses = []
      for i, (x_val, y_val) in enumerate(val_loader):
        # fix typing of labels
        y_val = y_val.type(torch.FloatTensor).unsqueeze(1)
        if torch.cuda.is_available():
          x_val = x_val.to(torch.device('cuda:0'))
          y_val = y_val.to(torch.device('cuda:0'))
        # forward step
        y_val_pred = self.forward(x_val)
        # get loss
        loss = self.criterion(y_val_pred, y_val)
        # get accuracy
        correct = (y_val_pred.round() == y_val).type(torch.float).sum().item()
        accuracy_batch = 100*correct/len(x_val)
        batch_accuracies.append(accuracy_batch)
        val_losses.append(loss.item())

        if (i+1 == n_val_steps):
          # get loss and accuracy
          avg_loss = sum(val_losses)/len(val_losses)
          accuracy = sum(batch_accuracies)/len(batch_accuracies)
          # add average loss to validation losses
          val_losses_epochs.append(avg_loss)
          print(f'Epoch {epoch+1}/{n_epochs} : Validation Accuracy: {accuracy:.2f}%, Validation Loss: {avg_loss:.4f}')



    return train_losses_epochs, val_losses_epochs, output

## Model Training

In [14]:
# init net
net = MyConvNet(lr=0.00005)
# use gpu if available
if torch.cuda.is_available():
  net = net.to(torch.device('cuda:0'))
# training loop
train_losses, val_losses, output = net.train(train_dataset, val_dataset, batch_size=10, n_epochs=8)
print('Finished Training')

OutOfMemoryError: ignored

### Training Visualization

In [ ]:
# plot the training and validation losses
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')

# add title and legend
plt.legend()
plt.title('Training and Validation Losses')

# display plot
plt.show()

## Model Evaluation